In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Extractor (Input Validation, Chunking, Mapping From file_csv and annotations_csv to chunks)

In [2]:
from acoustic_knowledge_discovery import extractors

/home/s.kamboj.400/acoustic_knowledge_discovery/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
extractor = extractors.Extractor(
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/file.csv", #file metadata csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/anno.csv", #annoations csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory",#input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
    chunk_size = 5) #chunk size in seconds
chunkDS = extractor.forward()


Casting the dataset: 100%|██████████| 6/6 [00:00<00:00, 849.65 examples/s]


In [4]:
for row in chunkDS.chunk_ds["train"]:
    print(row)
chunkDS.chunk_ds

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 10, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_10', 'Annotation': [], 'Confidence': []}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rai

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence'],
        num_rows: 6
    })
})

File-level Operations

In [5]:
from acoustic_knowledge_discovery.features.feature_preprocessing_pipeline import FeatureSequential
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6

pipeline = FeatureSequential(
    features.TemplateMatching(
        extractor.base_dir / "files",
        extractor.base_dir / "XC-templates",
        THRESHOLD,
        chunk_size=extractor.chunk_size
    ),
    # features.anotherFunctionThatInheretsFromFeaturePreprocessor
)

chunkDS = pipeline.forward(chunkDS)  

1 total templates
Filtered down to 1 unique species templates
  XC758442 - Common Blackbird - Turdus merula: XC758442 - Common Blackbird - Turdus merula.wav


Map: 100%|██████████| 6/6 [00:00<00:00, 748.14 examples/s]

Progress: 1/1 templates completed (100.0%)
Total matches found above the threshold of 0.6: 1


In [6]:
#verify properly loaded after feature extraction
for row in chunkDS.chunk_ds["train"]:
    print(row)

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 10, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis_10', 'Annotation': [], 'Confidence': []}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rai

Chunk-level Operations

In [12]:
#Insert EGCI

Canonical Graph Representation